In [88]:
import pandas as pd
from random import randint, choice, sample
from scipy.stats import binom
from numpy.random import choice
import numpy as np
from time import time as time_
from tqdm.notebook import tqdm
from copy import deepcopy

In [89]:
dt = pd.read_csv('forecast2.csv', sep=';')
fishs = ['щука', 'судак', 'окунь', 'берш', 'речная форель', 'озерная форель', 'елец', 'чехонь', 'сом', 'голавль', 'язь',
         'карп', 'жерех', 'лещ', 'карась', 'линь', 'пескарь', 'ротан', 'плотва', 'красноперка', 'налим', 'густера',
         'амур', 'ерш', 'сазан', 'подуст', 'толстолобик', 'вобла', 'хариус']
fishs = list(map(lambda x: x.capitalize(), fishs))
# dt = dt.rename(columns={fish: fish.capitalize() for fish in fishs})
dt

,day_temp,day_pressure,day_obl,day_phen,day_dir,day_wind,areal,city,year,month,...,Красноперка,Налим,Густера,Амур,Ерш,Сазан,Подуст,Толстолобик,Вобла,Хариус
0,-4,749,dull,snow,Ю,3,Алтайский край,Барнаул,2020,1,...,0.250000,0.357143,0.175000,0.133333,0.546875,0.192308,0.166667,0.178571,0.142857,0.160714
1,-1,750,dull,-,Ю,5,Алтайский край,Барнаул,2020,1,...,0.250000,0.285714,0.233333,0.133333,0.468750,0.192308,0.166667,0.178571,0.142857,0.160714
2,-3,749,dull,-,Ю,2,Алтайский край,Барнаул,2020,1,...,0.300000,0.357143,0.291667,0.166667,0.468750,0.230769,0.200000,0.178571,0.178571,0.214286
3,-6,753,dull,-,Ю,1,Алтайский край,Барнаул,2020,1,...,0.200000,0.285714,0.233333,0.166667,0.390625,0.192308,0.133333,0.142857,0.107143,0.160714
4,-2,752,suncl,-,Ю,3,Алтайский край,Барнаул,2020,1,...,0.300000,0.357143,0.408333,0.233333,0.390625,0.230769,0.200000,0.214286,0.214286,0.267857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30516,-6,752,dull,-,З,1,Московская область,Щелково,2020,12,...,0.208333,0.250000,0.233333,0.133333,0.468750,0.192308,0.166667,0.178571,0.142857,0.187500
30517,-6,759,sunc,-,ЮВ,1,Московская область,Щелково,2020,12,...,0.208333,0.312500,0.291667,0.200000,0.375000,0.230769,0.166667,0.178571,0.142857,0.250000
30518,-5,754,dull,-,ЮВ,3,Московская область,Щелково,2020,12,...,0.125000,0.125000,0.233333,0.133333,0.375000,0.115385,0.100000,0.107143,0.071429,0.125000
30519,-3,753,dull,-,-,0,Московская область,Щелково,2020,12,...,0.208333,0.250000,0.350000,0.200000,0.468750,0.192308,0.166667,0.178571,0.178571,0.250000


In [90]:
class DayGenerator:
    def __init__(self, dataframe):
        self.dataframe = dataframe
        self.times = [12, 15, 18, 21, 0, 3, 6, 9]
        self.dirs = {
            'С': {'values': ['СЗ', 'СВ'], 'p': [0.2, 0.3]},
            'СЗ': {'values': ['С', 'З'], 'p': [0.25, 0.25]},
            'З': {'values': ['СЗ', 'ЮЗ'], 'p': [0.2, 0.3]},
            'ЮЗ': {'values': ['З', 'Ю'], 'p': [0.25, 0.25]},
            'Ю': {'values': ['ЮЗ', 'ЮВ'], 'p': [0.3, 0.2]},
            'ЮВ': {'values': ['Ю', 'В'], 'p': [0.2, 0.3]},
            'В': {'values': ['СВ', 'ЮВ'], 'p': [0.3, 0.2]},
            'СВ': {'values': ['С', 'В'], 'p': [0.3, 0.2]},
        }
        
        self.phens = {
            'snow': {'values': ['небольшой снег', 'снег', 'снег с дождём', 'сильный снег', 'мокрый снег'], 'p': [0.35, 0.25, 0.15, 0.1, 0.15]},
            'rain': {'values': ['небольшой дождь', 'дождь', 'сильный дождь'], 'p': [0.45, 0.35, 0.2]},
            'storm': {'values': ['небольшой дождь', 'дождь', 'гроза', 'сильный дождь'], 'p': [0.15, 0.25, 0.3, 0.3]},
        }
        
        self.obl = {
            'dull': {'values': ['пасмурно', 'облачно'], 'p': [0.7, 0.3]},
            'suncl': {'values': ['облачно', 'малооблачно', 'пасмурно'], 'p': [0.6, 0.3, 0.1]},
            'sun': {'values': ['ясно', 'малооблачно'], 'p': [0.7, 0.3]},
            'sunc': {'values': ['малооблачно', 'ясно', 'облачно'], 'p': [0.6, 0.2, 0.2]},
        }
        
        self.fishs = ['Щука', 'Судак', 'Окунь', 'Берш', 'Речная форель', 'Озерная форель', 'Елец', 'Чехонь', 'Сом', 'Голавль', 'Язь',
         'Карп', 'Жерех', 'Лещ', 'Карась', 'Линь', 'Пескарь', 'Ротан', 'Плотва', 'Красноперка', 'Налим', 'Густера',
         'Амур', 'Ерш', 'Сазан', 'Подуст', 'Толстолобик', 'Вобла', 'Хариус']
        
    def __getitem__(self, idx):
        day_priew = self.dataframe.iloc[idx - 1]
        day = self.dataframe.iloc[idx]
        day_next = self.dataframe.iloc[idx + 1]
        pressure = ','.join(map(str, self.gen_pressure(day_priew['day_pressure'], day['day_pressure'], day_next['day_pressure'])))
        temperature = ','.join(map(str, self.gen_temperature(day_priew['day_temp'], day['day_temp'], day_next['day_temp'])))
        wind_, gust_ = self.gen_wind(day_priew['day_wind'], day['day_wind'], day_next['day_wind'])
        wind = ','.join(map(str, wind_))
        gust = ','.join(map(str, gust_))
        wind_direction = ','.join(self.gen_dir(day['day_dir']))
        phenomen_= self.gen_phenomen(day['day_obl'], day['day_phen'])
        humidity = ','.join(map(str,self.gen_hum(phenomen_)))
        phenomen = ','.join(phenomen_)
        uv_index = ','.join(map(str,self.gen_uv(day['month'])))
        moon_direction, moon = self.gen_moon(sum(day[fish].item() for fish in self.fishs) / len(self.fishs))
        return {
            'pressure': pressure,
            'temperature': temperature,
            'wind': wind,
            'gust': gust,
            'wind_direction': wind_direction,
            'humidity': humidity,
            'phenomenon': phenomen,
            'uv_index': uv_index,
            'moon_direction': moon_direction,
            'moon': moon,
            'month': day['month'].item(),
            'day': day['day'].item(),
            
        }
        
    def dist_(self, left, right, len_):
        is_revert = False
        if left > right:
            left, right = right, left
            is_revert = True
        result = []
        current = left
        for _ in range(len_ - 1): 
            sub = binom.rvs(right - current, 1 / len_)
            current += sub
            result.append(current)
        result.append(right)
        return result[::-1] if is_revert else result
        
    def gen_pressure(self, pressure_priew, pressure, pressure_next):
        pressure_ = {}
        priew_sub = pressure - pressure_priew
        next_sub = pressure_next - pressure
        current_pressure = pressure_priew
        for time in [15, 18, 21]:
            sub = randint(-2, 0) if priew_sub < 0 else randint(-1, 1)
            current_pressure += sub
        dist = self.dist_(current_pressure, pressure, 5)
        for idx, time in enumerate([0, 3, 6, 9, 12]):
            pressure_.update({time: dist[idx]})
        current_pressure = pressure
        for time in [15, 18, 21]:
            sub = randint(-2, 0) if next_sub < 0 else randint(-1, 1)
            current_pressure += sub
            pressure_.update({time: current_pressure})
        return [pressure_[key] for key in sorted(pressure_)]
    
    def gen_wind(self, wind_priew, wind, wind_next):
        wind_ = {}
        priew_sub = wind - wind_priew
        next_sub = wind_next - wind
        current_wind = randint(0, randint(wind_priew, wind)) if priew_sub > 0 else 0
        dist = self.dist_(current_wind, wind, 5)
        for idx, time in enumerate([0, 3, 6, 9, 12]):
            wind_.update({time: dist[idx]})
        current_wind = wind_[12] + randint(-1, 2)
        current_wind = max(0, current_wind)
        wind_.update({15: current_wind})
        current_wind += randint(-1, 2)
        current_wind = max(0, current_wind)
        wind_.update({18: current_wind})
        current_wind += randint(-3, 0)
        current_wind = max(0, current_wind)
        wind_.update({21: current_wind})
        gust_= {time: wind_[time] + randint(3, 8) for time in self.times}
        return [wind_[key] for key in sorted(wind_)], [gust_[key] for key in sorted(gust_)]
    
    def gen_dir(self,direction):
        dir_ = {}
        if direction == '-':
            direction = choice(list(self.dirs.keys()))
        dist = list(choice([direction] + self.dirs[direction]['values'], 8, p=[0.5] + self.dirs[direction]['p']))
        for idx, time in enumerate(self.times):
            dir_.update({time: dist[idx]})
        return [dir_[key] for key in sorted(dir_)]
    
    def gen_temperature(self, temp_priew, temp, temp_next):
        temp_ = {}
        priew_sub = temp - temp_priew
        next_sub = temp_next - temp
        current_temp = temp_priew
        for time in [15, 18, 21]:
            sub = randint(-3, -1) if priew_sub < 0 else randint(-2, 0)
            current_temp += sub
        dist = self.dist_(current_temp, temp, 5)
        for idx, time in enumerate([0, 3, 6, 9, 12]):
            temp_.update({time: dist[idx]})
        current_temp = temp
        for time in [15, 18, 21]:
            sub = randint(-3, -1) if next_sub < 0 else randint(-2, 0)
            current_temp += sub
            temp_.update({time: current_temp})
        return [temp_[key] for key in sorted(temp_)]
    
    def gen_phenomen(self, obl, phen):
        phens_ = {}
        if obl == '-':
            obl = 'sun'
        dist_olb = list(choice(self.obl[obl]['values'], 8, p=self.obl[obl]['p']))
        if phen == '-':
            for idx, time in enumerate(self.times):
                phens_.update({time: dist_olb[idx]})
            return [phens_[key] for key in sorted(phens_)] 
        dist_phens = list(choice(self.phens[phen]['values'], 8, p=self.phens[phen]['p']))
        for idx, time in enumerate(self.times):
            phens_.update({time: '.'.join((dist_olb[idx], dist_phens[idx]))})
        return [phens_[key] for key in sorted(phens_)]
    
    def gen_hum(self, phens):
        hum_ = []
        current_hum = randint(30, 70)
        for phen in phens:
            if 'дождь' in phen:
                current_hum += randint(0, 20)
                current_hum = min(current_hum, randint(93, 99))
            else:
                current_hum += randint(-10, 5)
                current_hum = max(current_hum, randint(10, 29))
            hum_.append(current_hum)
        return hum_
    
    def gen_uv(self, month):
        uv_ = {}
        if month in [12, 1, 2]:
            uv_ = {0: 0, 3: 0, 6: 0, 9: 0, 12: randint(1, 2), 15: 1, 18: 0, 21:0} 
        elif month in [3, 11, 10, 4]:
            uv_ = {0: 0, 3: 0, 6: 0, 9: 1, 12: randint(2, 3), 15: randint(1, 2), 18: 1, 21: 0}
        elif month in [5, 9]:
            uv_ = {0: 0, 3: 0, 6: randint(0, 1), 9: randint(1, 2), 12: randint(2, 4), 15: randint(2, 3), 18: randint(1, 2), 21: randint(0, 1)}
        else:
            uv_ = {0: 0, 3: 0, 6: randint(1, 2), 9: randint(1, 3), 12: randint(2, 4), 15: randint(1, 3), 18: randint(1, 3), 21: randint(1, 2)}
        return [uv_[key] for key in sorted(uv_)]
    
    def gen_moon(self, forecast):
        if forecast > 0.5:
            return 2* randint(0, 1) - 1, randint(35, 70)
        else:
            if randint(0, 1):
                return 2* randint(0, 1) - 1, randint(0, 35)
            else:
                return 2* randint(0, 1) - 1, randint(70, 99)   

In [91]:
ALONE_KEYS = {'time', 'day', 'month', 'humidity', 'uv_index', 'moon', 'moon_direction'}
DIGIT_KEYS = {'temperature', 'wind', 'gust', 'pressure', 'humidity', 'uv_index'}
CATEGORY_KEYS = {'phenomenon', 'wind_direction'}
MOON_KEYS = {'moon', 'moon_direction'}
SUN_KEYS = {'sun_up', 'sun_down'}
WIND_DIRECTIONS = ['Ю', 'ЮЗ', 'З', 'СЗ', 'С', 'СВ', 'В', 'ЮВ']
PHENOMENONS = ['ясно', 'малооблачно', 'облачно', 'пасмурно', 'небольшой дождь', 'дождь', 'сильный дождь',
               'небольшой снег', 'снег', 'снег с дождём', 'сильный снег', 'гроза', 'мокрый снег']
REGIONS = ['Алтайский край', 'Амурская область', 'Архангельская область', 'Астраханская область',
           'Белгородская область', 'Брянская область', 'Владимирская область', 'Волгоградская область',
           'Вологодская область', 'Воронежская область', 'Еврейская автономная область', 'Забайкальский край',
           'Ивановская область', 'Иркутская область', 'Кабардино-Балкарская республика', 'Калининградская область',
           'Калужская область', 'Камчатский край', 'Карачаево-Черкесская республика', 'Кемеровская область',
           'Кировская область', 'Костромская область', 'Краснодарский край', 'Красноярский край', 'Курганская область',
           'Курская область', 'Ленинградская область', 'Липецкая область', 'Магаданская область', 'Московская область']


num_hours = 8
num_days = 3

def preprocess_(data):
    all_data = {}
    for d in data:
        for key in DIGIT_KEYS:
            temp = list(map(int, d[key].split(',')))
            if key in all_data:
                all_data[key] = all_data[key] + temp
            else:
                all_data[key] = temp
        for key in MOON_KEYS:
            temp = [d[key] for _ in range(num_hours)]
            if key in all_data:
                all_data[key] = all_data[key] + temp
            else:
                all_data[key] = temp
        for key in CATEGORY_KEYS:
            if key in all_data:
                all_data[key] = all_data[key] + d[key].split(',')
            else:
                all_data[key] = d[key].split(',')
        days = [d['day'] for _ in range(num_hours)]
        months = [d['month'] for _ in range(num_hours)]
        if 'day' in all_data:
            all_data['day'] = all_data['day'] + days
        else:
            all_data['day'] = days
        if 'month' in all_data:
            all_data['month'] = all_data['month'] + months
        else:
            all_data['month'] = months
        if 'time' in all_data:
            all_data['time'] = all_data['time'] + list(range(0, num_hours * 3, 3))
        else:
            all_data['time'] = list(range(0, num_hours * 3, 3))
    return all_data

def slice_(data, left_bound, righ_bound):
    slice_data = {
        key: data[key][left_bound: righ_bound] for key in data
    }
    return slice_data

def preprocess_batch_(data):
    vec = {}
    for key in data:
        if key in DIGIT_KEYS and not key in ALONE_KEYS:
            for i in range(len(data[key])):
                key_name = '{}_{}'.format(key, i)
                vec.update({key_name: data[key][i]})
        elif key == 'phenomenon':
            phenomenons_ = [_.split('.') for _ in data[key]]
            for phenomenon in PHENOMENONS:
                for i in range(len(phenomenons_)):
                    key_name = '{}_{}'.format(phenomenon, i)
                    vec.update({key_name: int(phenomenon in phenomenons_[i])})
        elif key == 'wind_direction':
            for wind_direction in WIND_DIRECTIONS:
                for i in range(len(data[key])):
                    key_name = '{}_{}'.format(wind_direction, i)
                    vec.update({key_name: int(wind_direction == data[key][i])})
        elif key == 'month':
            for month in range(1, 13):
                key_name = 'month_{}'.format(month)
                vec.update({key_name: int(month == data[key][-1])})
        elif key == 'time':
            for time in range(0, 24, 3):
                key_name = 'time_{}'.format(time)
                vec.update({key_name: int(time == data[key][-1])})
        elif key == 'moon_direction':
            for moon_direction in [-1, 1]:
                key_name = 'moon_direction_{}'.format(moon_direction)
                vec.update({key_name: int(moon_direction == data[key][-1])})
        elif key in ALONE_KEYS:
            key_name = '{}'.format(key)
            vec.update({key_name: data[key][-1]})
    return vec

def gen_forecast(forecast, time, fish):
    if fish in ['Сом', 'Налим']:
        if time == 21:
            return forecast
        elif time == 6:
            return forecast
        elif time in [0, 3]:
            return min(1, forecast + np.random.uniform(0.1, 0.15))
        else:
            return max(0, forecast - np.random.uniform(0.2, 0.3))
    if time in [6, 18]:
        return min(1, forecast + np.random.uniform(0.1, 0.2))
    elif time == 9:
        return min(1, forecast + np.random.uniform(0.05, 0.1))
    elif time == 12:
        return min(1, forecast + np.random.uniform(0., 0.05))
    elif time == 15:
        return forecast
    else:
        return max(0, forecast - np.random.uniform(0.2, 0.3))
        
#     if fish in ['Сом', 'Налим']:
#         if time == 21:
#             return int(forecast) | (randint(1, 100) <= 20)
#         elif time in [0, 3]:
#             return int(forecast) | (randint(1, 100) <= 30)
#         elif time == 6:
#             return int(forecast) | (randint(1, 100) <= 20)
#         else:
#             return int(forecast) * (randint(1, 100) <= 40)
#     else:
#         if time in [6, 18]:
#             return int(forecast) | (randint(1, 100) <= 40)
#         elif time == 9:
#             return int(forecast) | (randint(1, 100) <= 30)
#         elif time == 12:
#             return int(forecast) | (randint(1, 100) <= 10)
#         elif time == 15:
#             return int(forecast)
#         else:
#             return int(forecast) * (randint(1, 100) <= 30)
            

In [92]:
gen = DayGenerator(dt)

In [93]:
init_ = False
train_dt = pd.DataFrame()
train_data = []
for idx, row in tqdm(dt.iterrows(), total=1300):
    if idx >= 1300:
        break
    if idx <= 3:
        continue
    if row['Щука'] > -1:
        data = dt[idx - 3: idx + 1]
        day = gen[idx]
        day_1 = gen[idx - 1]
        day_2 = gen[idx - 2]
        day_3 = gen[idx - 3]
        all_data = preprocess_([day_3, day_2, day_1, day])
        len_data = len(all_data['moon'])
        probs = {fish: [] for fish in gen.fishs}
        for i in range(num_hours * num_days, len_data + 1):
            slice_data = slice_(all_data, i - num_hours * num_days, i)
            vec = preprocess_batch_(slice_data)
            for fish in gen.fishs:
                for fish_ in gen.fishs:
                    vec.update({fish_: int(fish_ == fish)})
                time = 0
                for j in range(0, 24, 3):
                    if vec['time_{}'.format(j)] == 1:
                        time = j
                        break
                vec = {key: vec[key] for key in sorted(vec)}
                forecast = gen_forecast(row[fish], time, fish)
                vec.update({'forecast': forecast})
                train_data.append(deepcopy(vec))
        if not init_:
            train_dt = pd.DataFrame(train_data)
            train_data = []
            init_ = True
            continue
        train_dt = pd.concat([train_dt, pd.DataFrame(train_data)], ignore_index=True)
        train_data = []
        

In [95]:
train_dt.describe()

,day,gust_0,gust_1,gust_10,gust_11,gust_12,gust_13,gust_14,gust_15,gust_16,...,ясно_22,ясно_23,ясно_3,ясно_4,ясно_5,ясно_6,ясно_7,ясно_8,ясно_9,forecast
count,338256.000000,338256.000000,338256.000000,338256.000000,338256.000000,338256.000000,338256.000000,338256.000000,338256.000000,338256.000000,...,338256.000000,338256.000000,338256.000000,338256.000000,338256.000000,338256.000000,338256.000000,338256.000000,338256.000000,338256.000000
mean,15.714506,7.927555,7.966564,7.985168,8.005573,8.112912,8.174040,8.235340,8.103567,7.939643,...,0.252486,0.253772,0.249228,0.251200,0.251972,0.254287,0.253944,0.252572,0.253858,0.280874
std,8.775345,2.907674,2.900724,2.867004,2.863177,2.893494,2.926937,2.973063,2.937378,2.883598,...,0.434439,0.435169,0.432567,0.433704,0.434146,0.435460,0.435266,0.434488,0.435218,0.229686
min,1.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.072413
50%,16.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.263255
75%,23.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.428571
max,31.000000,24.000000,24.000000,22.000000,22.000000,22.000000,22.000000,23.000000,23.000000,23.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [102]:
train_dt.to_csv('train_dt.csv', sep=';', index=False)

In [96]:
# train_dt = pd.DataFrame(train_data)

In [97]:
# fishs_dt = [train_dt[train_dt[fish.capitalize()] == 1] for fish in fishs]

In [98]:
# y_means = {fishs[i]: fishs_dt[i]['forecast'].mean() for i in range(len(fishs))}
# y_means

In [99]:
# f1 = train_dt[train_dt['forecast'] == 1]
# f0 = train_dt[train_dt['forecast'] == 0].sample(len(f1))
# train_dt = pd.concat([f0, f1])

In [100]:
# good_idx = np.where(train_dt['forecast'] == 1)[0]
# bad_idx = np.where(train_dt['forecast'] == 0)[0]
# train_dt = pd.concat([train_dt.loc[list(good_idx), :], train_dt.loc[sample(list(bad_idx), len(good_idx)), :]])

In [101]:
# train_dt[fishs].describe() * len(fishs)

In [103]:
train_dt = train_dt.sample(frac=0.3).reset_index(drop=True)

In [104]:
y = train_dt['forecast']
del train_dt['forecast']

In [105]:
y.std()

0.22981340359448707

In [106]:
X = train_dt.values

In [107]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [108]:
X_train.shape

(81181, 655)

In [109]:
X_test.shape

(20296, 655)

In [ ]:
from catboost import CatBoostRegressor

model = CatBoostRegressor()
model.fit([[1, 2], [3, 4]], [1, 4])
model.predict([1, 2])

In [111]:
model.fit(X_train, y_train)

Learning rate set to 0.085366
0:	learn: 0.2207598	total: 35.5ms	remaining: 35.4s
1:	learn: 0.2127212	total: 62.6ms	remaining: 31.2s
2:	learn: 0.2055980	total: 87.9ms	remaining: 29.2s
3:	learn: 0.1992009	total: 115ms	remaining: 28.6s
4:	learn: 0.1936009	total: 139ms	remaining: 27.6s
5:	learn: 0.1887566	total: 168ms	remaining: 27.8s
6:	learn: 0.1845062	total: 193ms	remaining: 27.3s
7:	learn: 0.1804900	total: 220ms	remaining: 27.3s
8:	learn: 0.1764225	total: 247ms	remaining: 27.2s
9:	learn: 0.1732303	total: 272ms	remaining: 26.9s
10:	learn: 0.1702505	total: 298ms	remaining: 26.8s
11:	learn: 0.1673113	total: 321ms	remaining: 26.5s
12:	learn: 0.1648003	total: 350ms	remaining: 26.6s
13:	learn: 0.1619488	total: 378ms	remaining: 26.6s
14:	learn: 0.1600206	total: 404ms	remaining: 26.6s
15:	learn: 0.1584361	total: 429ms	remaining: 26.4s
16:	learn: 0.1561007	total: 458ms	remaining: 26.5s
17:	learn: 0.1544541	total: 481ms	remaining: 26.2s
18:	learn: 0.1530416	total: 507ms	remaining: 26.2s
19:	lear

163:	learn: 0.1103403	total: 4.17s	remaining: 21.2s
164:	learn: 0.1102041	total: 4.19s	remaining: 21.2s
165:	learn: 0.1100740	total: 4.22s	remaining: 21.2s
166:	learn: 0.1099760	total: 4.25s	remaining: 21.2s
167:	learn: 0.1098643	total: 4.27s	remaining: 21.2s
168:	learn: 0.1097412	total: 4.3s	remaining: 21.1s
169:	learn: 0.1096583	total: 4.32s	remaining: 21.1s
170:	learn: 0.1095479	total: 4.36s	remaining: 21.1s
171:	learn: 0.1094761	total: 4.39s	remaining: 21.1s
172:	learn: 0.1093998	total: 4.42s	remaining: 21.1s
173:	learn: 0.1092370	total: 4.44s	remaining: 21.1s
174:	learn: 0.1091355	total: 4.47s	remaining: 21.1s
175:	learn: 0.1090168	total: 4.49s	remaining: 21s
176:	learn: 0.1088910	total: 4.52s	remaining: 21s
177:	learn: 0.1087988	total: 4.54s	remaining: 21s
178:	learn: 0.1086846	total: 4.56s	remaining: 20.9s
179:	learn: 0.1085849	total: 4.59s	remaining: 20.9s
180:	learn: 0.1084860	total: 4.61s	remaining: 20.9s
181:	learn: 0.1084025	total: 4.64s	remaining: 20.8s
182:	learn: 0.10830

326:	learn: 0.0983836	total: 8.53s	remaining: 17.5s
327:	learn: 0.0983242	total: 8.57s	remaining: 17.6s
328:	learn: 0.0982790	total: 8.61s	remaining: 17.6s
329:	learn: 0.0982231	total: 8.64s	remaining: 17.5s
330:	learn: 0.0981485	total: 8.68s	remaining: 17.5s
331:	learn: 0.0980971	total: 8.72s	remaining: 17.5s
332:	learn: 0.0980422	total: 8.75s	remaining: 17.5s
333:	learn: 0.0980064	total: 8.78s	remaining: 17.5s
334:	learn: 0.0979475	total: 8.82s	remaining: 17.5s
335:	learn: 0.0979048	total: 8.85s	remaining: 17.5s
336:	learn: 0.0978495	total: 8.89s	remaining: 17.5s
337:	learn: 0.0977940	total: 8.93s	remaining: 17.5s
338:	learn: 0.0977420	total: 8.96s	remaining: 17.5s
339:	learn: 0.0976371	total: 9s	remaining: 17.5s
340:	learn: 0.0975885	total: 9.04s	remaining: 17.5s
341:	learn: 0.0975426	total: 9.08s	remaining: 17.5s
342:	learn: 0.0974972	total: 9.12s	remaining: 17.5s
343:	learn: 0.0974509	total: 9.16s	remaining: 17.5s
344:	learn: 0.0973865	total: 9.2s	remaining: 17.5s
345:	learn: 0.09

485:	learn: 0.0916340	total: 14.4s	remaining: 15.2s
486:	learn: 0.0916078	total: 14.4s	remaining: 15.2s
487:	learn: 0.0915658	total: 14.5s	remaining: 15.2s
488:	learn: 0.0915347	total: 14.5s	remaining: 15.2s
489:	learn: 0.0915072	total: 14.6s	remaining: 15.1s
490:	learn: 0.0914840	total: 14.6s	remaining: 15.1s
491:	learn: 0.0914540	total: 14.6s	remaining: 15.1s
492:	learn: 0.0914292	total: 14.7s	remaining: 15.1s
493:	learn: 0.0914017	total: 14.7s	remaining: 15.1s
494:	learn: 0.0913647	total: 14.8s	remaining: 15.1s
495:	learn: 0.0913408	total: 14.8s	remaining: 15.1s
496:	learn: 0.0912698	total: 14.9s	remaining: 15s
497:	learn: 0.0912347	total: 14.9s	remaining: 15s
498:	learn: 0.0911739	total: 14.9s	remaining: 15s
499:	learn: 0.0911440	total: 15s	remaining: 15s
500:	learn: 0.0911142	total: 15s	remaining: 15s
501:	learn: 0.0910754	total: 15s	remaining: 14.9s
502:	learn: 0.0910473	total: 15.1s	remaining: 14.9s
503:	learn: 0.0910242	total: 15.1s	remaining: 14.9s
504:	learn: 0.0909963	total:

648:	learn: 0.0869450	total: 20.4s	remaining: 11.1s
649:	learn: 0.0869184	total: 20.5s	remaining: 11s
650:	learn: 0.0868972	total: 20.5s	remaining: 11s
651:	learn: 0.0868741	total: 20.6s	remaining: 11s
652:	learn: 0.0868459	total: 20.6s	remaining: 10.9s
653:	learn: 0.0868250	total: 20.6s	remaining: 10.9s
654:	learn: 0.0868083	total: 20.7s	remaining: 10.9s
655:	learn: 0.0867786	total: 20.7s	remaining: 10.9s
656:	learn: 0.0867379	total: 20.7s	remaining: 10.8s
657:	learn: 0.0867149	total: 20.8s	remaining: 10.8s
658:	learn: 0.0866945	total: 20.8s	remaining: 10.8s
659:	learn: 0.0866792	total: 20.8s	remaining: 10.7s
660:	learn: 0.0866554	total: 20.9s	remaining: 10.7s
661:	learn: 0.0866261	total: 20.9s	remaining: 10.7s
662:	learn: 0.0866079	total: 20.9s	remaining: 10.6s
663:	learn: 0.0865808	total: 21s	remaining: 10.6s
664:	learn: 0.0865680	total: 21s	remaining: 10.6s
665:	learn: 0.0865469	total: 21.1s	remaining: 10.6s
666:	learn: 0.0864875	total: 21.1s	remaining: 10.5s
667:	learn: 0.0864608	

811:	learn: 0.0833551	total: 26.4s	remaining: 6.1s
812:	learn: 0.0833234	total: 26.4s	remaining: 6.07s
813:	learn: 0.0833017	total: 26.4s	remaining: 6.04s
814:	learn: 0.0832936	total: 26.5s	remaining: 6.01s
815:	learn: 0.0832671	total: 26.5s	remaining: 5.98s
816:	learn: 0.0832501	total: 26.5s	remaining: 5.94s
817:	learn: 0.0832260	total: 26.6s	remaining: 5.91s
818:	learn: 0.0832043	total: 26.6s	remaining: 5.88s
819:	learn: 0.0831902	total: 26.6s	remaining: 5.85s
820:	learn: 0.0831745	total: 26.7s	remaining: 5.82s
821:	learn: 0.0831548	total: 26.7s	remaining: 5.78s
822:	learn: 0.0831394	total: 26.7s	remaining: 5.75s
823:	learn: 0.0831233	total: 26.8s	remaining: 5.72s
824:	learn: 0.0831108	total: 26.8s	remaining: 5.69s
825:	learn: 0.0830897	total: 26.8s	remaining: 5.65s
826:	learn: 0.0830667	total: 26.9s	remaining: 5.62s
827:	learn: 0.0830496	total: 26.9s	remaining: 5.59s
828:	learn: 0.0830142	total: 27s	remaining: 5.56s
829:	learn: 0.0829931	total: 27s	remaining: 5.53s
830:	learn: 0.082

974:	learn: 0.0804398	total: 32.3s	remaining: 829ms
975:	learn: 0.0804286	total: 32.4s	remaining: 796ms
976:	learn: 0.0804031	total: 32.4s	remaining: 763ms
977:	learn: 0.0803802	total: 32.4s	remaining: 729ms
978:	learn: 0.0803668	total: 32.5s	remaining: 696ms
979:	learn: 0.0803575	total: 32.5s	remaining: 663ms
980:	learn: 0.0803429	total: 32.5s	remaining: 630ms
981:	learn: 0.0803353	total: 32.6s	remaining: 597ms
982:	learn: 0.0803243	total: 32.6s	remaining: 564ms
983:	learn: 0.0803089	total: 32.6s	remaining: 531ms
984:	learn: 0.0802930	total: 32.7s	remaining: 498ms
985:	learn: 0.0802806	total: 32.7s	remaining: 465ms
986:	learn: 0.0802517	total: 32.7s	remaining: 431ms
987:	learn: 0.0802324	total: 32.8s	remaining: 398ms
988:	learn: 0.0802187	total: 32.8s	remaining: 365ms
989:	learn: 0.0801991	total: 32.9s	remaining: 332ms
990:	learn: 0.0801823	total: 32.9s	remaining: 299ms
991:	learn: 0.0801639	total: 32.9s	remaining: 266ms
992:	learn: 0.0801439	total: 33s	remaining: 232ms
993:	learn: 0.

In [112]:
preds = model.predict(X_test)

In [113]:
from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(preds, y_test))

0.06502638494944578


In [115]:
model.predict(X_test)[:100]

array([ 4.78229100e-01, -4.30534714e-04,  4.09222604e-02,  3.64393460e-01,
        9.35556748e-02,  4.43562532e-01,  3.83472171e-01,  4.86374582e-01,
        4.23233036e-01,  2.16796563e-01,  1.51323229e-01,  2.34430456e-02,
        1.21307776e-01,  5.79526730e-01,  3.46541635e-01,  6.12075862e-01,
        2.75068789e-01,  5.61137071e-01,  5.11210202e-01, -5.47035898e-02,
        4.44026908e-01,  4.28060087e-01,  1.04264584e-01,  2.76546681e-01,
       -5.24961092e-02,  6.89232647e-02,  5.16256344e-01, -1.23175641e-02,
        3.54245592e-01,  1.04120954e-01,  8.90884269e-02,  2.28861825e-01,
        4.93613202e-01,  2.42369282e-01,  1.91198141e-01,  3.73853167e-01,
        4.46304577e-01,  3.39047883e-01,  2.95841223e-01,  2.60432901e-01,
        1.68366379e-01,  5.16450403e-01,  5.94345300e-01,  3.53003621e-01,
        9.78342995e-02, -3.41581889e-02, -1.84923408e-02,  4.79698368e-01,
        2.72113865e-01,  1.46600305e-01,  1.51870428e-01,  1.23631383e-01,
       -1.22601631e-02, -

In [116]:
from joblib import dump

In [117]:
dump(model, 'catboost_0.6.model')

['catboost_0.6.model']

In [118]:
y_test.mean()

0.2809809231832194

In [119]:
dt_test = pd.DataFrame(X_test, columns=train_dt.columns)
dt_test['forecast'] = preds
dt_test

,day,gust_0,gust_1,gust_10,gust_11,gust_12,gust_13,gust_14,gust_15,gust_16,...,ясно_22,ясно_23,ясно_3,ясно_4,ясно_5,ясно_6,ясно_7,ясно_8,ясно_9,forecast
0,24,6,7,8,7,7,10,10,10,9,...,0,0,0,0,0,0,0,0,0,0.478229
1,12,4,4,9,5,5,7,6,7,9,...,1,0,1,1,1,0,0,1,1,-0.000431
2,20,6,7,11,12,16,12,16,8,6,...,0,0,0,0,0,0,0,0,0,0.040922
3,31,11,8,7,7,6,7,6,6,4,...,0,0,0,0,0,0,0,0,0,0.364393
4,19,9,5,7,10,7,8,8,5,8,...,0,0,0,0,0,0,0,0,0,0.093556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20291,24,9,8,5,8,8,9,10,11,14,...,0,0,0,0,0,0,0,0,1,0.308302
20292,24,9,8,4,7,6,9,6,6,8,...,0,0,0,0,0,0,0,0,0,0.269273
20293,13,8,8,9,5,8,10,10,12,9,...,1,0,0,0,0,0,0,0,0,0.194469
20294,21,9,6,10,13,4,8,4,6,7,...,0,0,0,0,0,1,1,1,0,0.762593
